## Imports

In [80]:
import os

import spacy
import random
import json
import numpy as np

from sklearn.model_selection import train_test_split
from spacy.training.example import Example

In [81]:
DATA_PATH = os.path.abspath("../../data/datasets/ner-swiss-receipts.json")  # Path to your training data file
OUTPUT_DIR = os.path.abspath("../models/receipts-ner-v02")  # TODO: make sure to use a new version
ITERATIONS = 30  # Number of training iterations
BATCH_SIZE = 8  # Batch size for training
DROPOUT = 0.5  # Dropout rate

## Preprocessing

### Load Dataset

In [82]:
with open(DATA_PATH, "r", encoding="utf-8") as file:
    dataset = json.load(file)

### Reformat for Spacy & Remove unwanted Entities

In [83]:
data = []
for entry in dataset:
    text = entry["text"]
    entities = [entity for entity in entry["entities"] if entity[2] == 'RECEIPT_ITEM']
    data.append((text, {"entities": entities}))

### Split Data into Train & Test

In [84]:
train, test = train_test_split(data, test_size=0.1)

In [85]:
print("Sample Training Data:")
sample_text, sample_annotaiton = train[0]
print("-----------------------------------------")
print(sample_text)
print("-----------------------------------------")
for ent in sample_annotaiton.get("entities"):
    print(f'Entity-Type: {ent[2]}, Entity text: {sample_text[ent[0]:ent[1]]}')

Sample Training Data:
-----------------------------------------
HAMNUGLU GmbH
HERTENSTEIN

Hertensteinstrasse 32

6004 Luzern

Tel 041 419 71 51
RER

#CS0 Rechnungsnummer: 36 POSEJ Order: el
KASSE 32- 2071272024 14:23:55

ANZ ARTIKEL GESAMT "
1MeChicken 6.90 M
ı MM 9 McNuggets f 15.800. Ep
19 Nucpets 2
1 Cocktail Sauce

. 1 Potatoes Sauce
1 Hen ohne Ksäure 0.5 2
1 Medium Pommes Frite ed
1 Cocktail Sauce en

Subtotal 2.10 u
Discount: i -1.30 Be
INNEN TOTAL 21.40 wo
VISA 210 0

st.Nr. CHE - 449.055.085 MWST ] 0
SATZ BRUTTO MWST.
incl. MWST  8.10 21.40 1.60

 Kundenbeleg

Buchung
. Visa payWave contact less

4323 22XX XXXX 1361

20.12.2024 14:23:41
u Trm-Id: 10138349
AID: 40000000031010
Irx. Seg-Cnt: Mo

..... Trx. Ref-No: .

Auth, Code:

-----------------------------------------
Entity-Type: RECEIPT_ITEM, Entity text: 1MeChicken 6.90 M
Entity-Type: RECEIPT_ITEM, Entity text: ı MM 9 McNuggets f 15.800. Ep


## Model Definition

Load an emtpy spacy model for specific language

In [86]:
# For a blank model, or replace with spacy.load("en_core_web_sm")
nlp = spacy.blank("de")

### Add Named-Entity Recognition Component

In [87]:
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

### Add Labels to Model

In [88]:
for _, annotations in train:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

### Spacy Examples

In [89]:
examples = [
    Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in train
]

### Training

In [90]:
# Disable other pipes during training for efficiency
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
        
    # Training loop
    print("Starting training...")
    for epoch in range(ITERATIONS):
        random.shuffle(examples)
        losses = {}
        for batch in spacy.util.minibatch(examples, size=BATCH_SIZE):
            nlp.update(batch, drop=DROPOUT, losses=losses)
        print(f"Epoch {epoch + 1}/{ITERATIONS}, Loss: {losses['ner']:.4f}")

Starting training...
Epoch 1/30, Loss: 5313.0225
Epoch 2/30, Loss: 1999.4539
Epoch 3/30, Loss: 1437.6208
Epoch 4/30, Loss: 1399.8524
Epoch 5/30, Loss: 1038.3199
Epoch 6/30, Loss: 769.4518
Epoch 7/30, Loss: 723.7105
Epoch 8/30, Loss: 651.6862
Epoch 9/30, Loss: 557.2901
Epoch 10/30, Loss: 464.8676
Epoch 11/30, Loss: 406.5923
Epoch 12/30, Loss: 352.5955
Epoch 13/30, Loss: 326.8011
Epoch 14/30, Loss: 249.9826
Epoch 15/30, Loss: 251.2966
Epoch 16/30, Loss: 206.3507
Epoch 17/30, Loss: 217.6507
Epoch 18/30, Loss: 181.3991
Epoch 19/30, Loss: 183.8387
Epoch 20/30, Loss: 173.1094
Epoch 21/30, Loss: 146.1970
Epoch 22/30, Loss: 160.8596
Epoch 23/30, Loss: 140.8577
Epoch 24/30, Loss: 128.4356
Epoch 25/30, Loss: 122.5660
Epoch 26/30, Loss: 107.2497
Epoch 27/30, Loss: 130.2041
Epoch 28/30, Loss: 108.4909
Epoch 29/30, Loss: 95.8912
Epoch 30/30, Loss: 96.8432


### Save Model

In [91]:
# Save the trained model
print(f"Saving model to {OUTPUT_DIR}...")
nlp.to_disk(OUTPUT_DIR)

Saving model to /Users/timon/git/dspro1-receipt-ocr/named-entity-recognition/models/receipts-ner-v02...


## Evaluation

In [92]:
def print_confusion_matrix(matrix, labels):
    """
    Prints a confusion matrix as a formatted string.

    Args:
        matrix (list of list of int): Confusion matrix as a 2D list.
        labels (list of str): Labels for the matrix.
    """
    max_label_length = max(len(label) for label in labels)
    padding = max_label_length + 2

    # Print header
    header = " " * padding + " ".join(f"{label:>{padding}}" for label in labels)
    print(f'{"":<15} Predicted Values')
    print(header)

    # Print each row
    for label, row in zip(labels, matrix):
        row_str = " ".join(f"{cell:>{padding}}" for cell in row)
        print(f"{label:>{padding}} {row_str}")

In [93]:
true_positive = []
false_positive = []
false_negative = []

index = 0

for text, annotations in test:
    # Process the text
    doc = nlp(text)
    
    # Prepare expected values (y)
    expected_entities = []
    for ent in annotations.get("entities"):    
        if ent[2] == 'RECEIPT_ITEM':
            expected_entities.append(text[ent[0]:ent[1]])

    # Print recognized entities
    for ent in doc.ents:
        if ent.text in expected_entities:
            true_positive.append(ent.text)
            expected_entities.remove(ent.text)
        else:
            false_positive.append(ent.text)
            
    false_negative.extend(expected_entities)
    
# Example usage
labels = ["Entity", "No Entity"]
confusion_matrix = [
    [len(true_positive), len(false_negative)],
    [len(false_positive), '0'],
]

print('Confusion Matrix:')
print_confusion_matrix(confusion_matrix, labels)

print('\nPerformance Measures:')
precision = len(true_positive) / (len(true_positive) + len(false_positive))
recall = len(true_positive) / (len(true_positive) + len(false_negative))
f1_score = 2 * precision * recall / (precision + recall)
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('F1 Score: ' + str(f1_score))

# True-Positive = correct idetified entities
# false_positive = actual entity that was not identified
# False-negative = identified entity that is actually not a entity

Confusion Matrix:
                Predicted Values
                Entity   No Entity
     Entity          26           5
  No Entity           7           0

Performance Measures:
Precision: 0.7878787878787878
Recall: 0.8387096774193549
F1 Score: 0.8125
